In [64]:
import numpy as np
import pandas as pd
from collections import Counter
from math import ceil, floor
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import os
from argparse import Namespace

import ast

from gensim.models import Word2Vec

flags = Namespace(
    train_file='dv9ogm_corpus.txt',
    seq_size=8,
    batch_size=64,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['<c>', '<s>'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
) 

In [ ]:
class SubmissionCorpusDataset(Dataset):
    def __init__(self, corpus, root_dir):
        self.corpus = corpus
        self.judgement_categories = corpus.keys()
        self.vocabulary_size
        self.embedding_size
        
        all_comments = []
        all_words = []
        for judgement_category in judgement_categories:
            for (comment_score, comment_body) in corpus[judgement_category]:
                all_comments.append(comment_body)
                for token in comment_body:
                    all_words.append(token)
        self.w2v = Word2Vec(all_comments, min_count=1))
        
        #word_counts = Counter(all_words)
        #sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
        #int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
        #vocab_to_int = {w: k for k, w in int_to_vocab.items()}
        #n_vocab = len(int_to_vocab)

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [65]:
def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r') as f:
        text_dict = f.read()
    
    corpus = ast.literal_eval(text_dict)
    judgement_categories = corpus.keys()
    all_comments = []
    all_words = []
    for judgement_category in judgement_categories:
        for (comment_score, comment_body) in corpus[judgement_category]:
            all_comments.append(comment_body)
            for token in comment_body:
                all_words.append(token)
    
    word_counts = Counter(all_words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)
    
    int_text = [vocab_to_int[w] for w in all_words]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return(int_to_vocab, vocab_to_int, n_vocab, in_text, out_text)

In [66]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield(in_text[:, i:i+seq_size], out_text[:, i:i+seq_size])

In [67]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return(logits, state)
    
    def zero_state(self, batch_size):
        return(torch.zeros(1, batch_size, self.lstm_size),
               torch.zeros(1, batch_size, self.lstm_size))

In [68]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return(criterion, optimizer)

In [59]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [71]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0
    
    
    for e in range(500):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        
        # Transfer data to GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            
            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            # Transfer data to GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # Perform back-propagation
            loss.backward(retain_graph=True)

            # Update the network's parameters
            optimizer.step()
            
            loss.backward()

            _ = torch.nn.utils.clip_grad_norm_(
                net.parameters(), flags.gradients_norm)

            optimizer.step()
            
            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                torch.save(net.state_dict(),
                           'checkpoint_pt/model-{}.pth'.format(iteration))
    
    #predict(device, net, flags.initial_words, n_vocab, vocab_to_int, int_to_vocab, top_k=5)
                

In [72]:
main()

Epoch: 14/200 Iteration: 100 Loss: 0.9190024137496948
Epoch: 28/200 Iteration: 200 Loss: 0.30645227432250977
Epoch: 42/200 Iteration: 300 Loss: 0.16847598552703857
Epoch: 57/200 Iteration: 400 Loss: 0.0689917802810669
Epoch: 71/200 Iteration: 500 Loss: 0.024868033826351166
Epoch: 85/200 Iteration: 600 Loss: 0.36516422033309937
Epoch: 99/200 Iteration: 700 Loss: 0.034282926470041275
Epoch: 114/200 Iteration: 800 Loss: 0.02150796540081501
Epoch: 128/200 Iteration: 900 Loss: 0.016010718420147896
Epoch: 142/200 Iteration: 1000 Loss: 0.007311515975743532
<c> <s> YTA , your desire to it does to do your parents body and her her body with your body with your whole time makes me wonder are you do n't need a favor straight to `` do believe you can afford try 's behaving thing all ) '' . and I 'm not tried right , producing it . matter my bump to 'd produce surrogacy confined his sister is inappropriate to do you were a vegetarian wanting asking a HELL ask suggests knew you would even understand 

In [53]:
'''
class SubmissionCorpusDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, corpus, root_dir):
        self.corpus = corpus
        self.judgement_categories = corpus.keys()
        self.vocabulary_size
        self.embedding_size
        
    def build_word2vec(self):
        all_comments = []
        all_words = []
        category_inds = np.arange(len(self.judgements))
        for category_ind in category_inds:
            for comment_ind in np.arange(len(self.comment_corpus[self.judgements[category_ind]])):
                all_comments.append(self.comment_corpus[self.judgements[category_ind]][comment_ind])
                #for token in self.comment_corpus[self.judgements[category_ind]][comment_ind]:
                #    all_words.append(token)

        return(Word2Vec(all_comments, min_count=1))

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample
'''

'\nclass SubmissionCorpusDataset(Dataset):\n    """Face Landmarks dataset."""\n\n    def __init__(self, corpus, root_dir):\n        self.corpus = corpus\n        self.judgement_categories = corpus.keys()\n        self.vocabulary_size\n        self.embedding_size\n        \n    def build_word2vec(self):\n        all_comments = []\n        all_words = []\n        category_inds = np.arange(len(self.judgements))\n        for category_ind in category_inds:\n            for comment_ind in np.arange(len(self.comment_corpus[self.judgements[category_ind]])):\n                all_comments.append(self.comment_corpus[self.judgements[category_ind]][comment_ind])\n                #for token in self.comment_corpus[self.judgements[category_ind]][comment_ind]:\n                #    all_words.append(token)\n\n        return(Word2Vec(all_comments, min_count=1))\n\n    def __len__(self):\n        return len(self.landmarks_frame)\n\n    def __getitem__(self, idx):\n        if torch.is_tensor(idx):\n       